# Лабораторная работа 6. Вариант 1. Группа 343

## Задание

Написать программу, решающую задачу указанным проекционным методом и методом коллокации.



## Условие

1) Граничная задача
$-(\frac{1}{2+x}u')'+cos(x)u=1+x, u(-1)=u(1)=0$

2) Метод Ритца

   Координатная система: $(1-x^2)P_i^{(1,1)}(x),i=0,1,...$
   
3) Метод коллокации

Исходя из условия:

$\alpha_1=1$

$\alpha_2=0$

$\beta_1=1$

$\beta_2=0$

Как будет происходить дифференциирование?

$[(1-x^2)P_i^{(1,1)}(x)]'=-2(i+1)P_{i+1}^{(0,0)}(x)$

$[P_i^{(0,0)}(x)]'=\frac{i+1}{2}P_{i-1}^{(1,1)}(x)$

$[P_{i+1}^{(0,0)}(x)]'=\frac{i+2}{2}P_i^{(1,1)}(x)$

$[(1-x^2)P_i^{(1,1)}(x)]''= [-2(i+1)P^{(0,0)}_{i+1}(x)]'=$

$=-2(i+1)[P^{(0,0)}_{i+1}(x)]'=-(i+1)(i+2)P_i^{(1, 1)}(x)$

In [1]:
import numpy as np
from scipy.integrate import quad
from numpy import linalg as LA
from tabulate import tabulate

N_RANGE = (3, 7)
ALPHA_1 = 1
ALPHA_2 = 0
BETA_1 = 1
BETA_2 = 0


def find_solution_gauss(A, b, use_pivoting=False):
    n = len(A)

    for i in range(n - 1):
        if use_pivoting:
            pivot_index = abs(A[i:, i]).argmax() + i

            if pivot_index != i:
                A[[i, pivot_index]] = A[[pivot_index, i]]
                b[[i, pivot_index]] = b[[pivot_index, i]]

        for j in range(i + 1, n):
            ratio = A[j, i] / A[i, i]
            A[j, i:] -= ratio * A[i, i:]
            b[j] -= ratio * b[i]

    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]

    return x


def find_cond(A):
    norm_a = LA.norm(A, np.inf)
    inverse_a = LA.inv(A)
    norm_inverse_a = LA.norm(inverse_a, np.inf)
    return norm_a * norm_inverse_a


def integrate(f, a, b):
    return quad(f, a, b)[0]


# Required functions:
# Equation: -(p(x)u')' + r(x)u = f(x)
# => p(x) = 1/(2+x)
# => r(x) = cos(x)
# => f(x) = 1+x

def p(x):
    return 1 / (2 + x)


def dp(x):
    return -1 / (2 + x) ** 2


def r(x):
    return np.cos(x)


def f(x):
    return 1 + x


def q(x):
    return 0

# Calculate Jacobi polynom 
def P(n, k):
    def polynom(x):
        if n == 0:
            return 1
        if n == 1:
            return (k + 1) * x
        
        num = (n + k + 2) * (2 * n + 2 * k + 3) * x * P(n - 1, k)(x)
        num -= (n + k + 2) * (n + k + 1) * P(n - 2, k)(x)
        denum = (n + 2 * k + 2) * (n + 2)
        return num / denum
        
    return polynom
    

def w_i(i):
    return lambda x: (1 - x ** 2) * P(i, 1)(x)
        

def dw_i(i):
    return lambda x: -2 * (i + 1) * P(i + 1, 0)(x)


def d2w_i(i):
    return lambda x: - (i + 1) * (i + 2) * P(i, 1)(x)


def q_l(y, z):
    # I, II (ALPHA_2 = 0 here)
    if ALPHA_2 == 0 or ALPHA_1 == 0:
        return 0
    # III (both not 0)
    return ALPHA_1 / ALPHA_2 * p(-1) * y(-1) * z(-1)


def q_r(y, z):
    # I, II (BETA_2 = 0 here)
    if BETA_2 == 0 or BETA_1 == 0:
        return 0
    # III
    return BETA_1 / BETA_2 * p(1) * y(1) * z(1)


def f_integrated(y, z, dy, dz):
    return lambda x: p(x) * dy(x) * dz(x) + r(x) * y(x) * z(x)

In [2]:
def find_A_ritz(n):
    A = np.zeros((n, n))
    # find a_ij as [w_i, w_j]
    for i in range(0, n):
        for j in range(0, n):
            wi = w_i(i)
            wj = w_i(j)
            dwi = dw_i(i)
            dwj = dw_i(j)
            integral = integrate(f_integrated(wi, wj, dwi, dwj), -1, 1)
            A[i, j] = integral + q_l(wi, wj) + q_r(wi, wj)
    
    return A


def find_solution_ritz(A, x, n):
    fs = []
    for i in range(0, n):
        # f_i: (f, w_i) -> integrate f*w_i from -1 to 1
        fs.append(integrate(lambda x: w_i(i)(x) * f(x), -1, 1))
        
    cs = find_solution_gauss(np.copy(A), np.copy(fs))
    
    res = 0
    for i in range(0, n):
        res += cs[i] * w_i(i)(x)
        
    return res

In [3]:
def find_solution_collocation(x, n):
    ts = []
    for i in range(1, n + 1):
        ts.append(np.cos((2 * i - 1) * np.pi / (2 * n)))
    
    def lu(i, t):
        return -(p(t) * d2w_i(i)(t) + dp(t) * dw_i(i)(t)) + q(t) * dw_i(i)(t) + r(t) * w_i(i)(t)
    
    A = np.zeros((n, n))
    for i in range(0, n):
        for j in range(0, n):
            A[i, j] = lu(j, ts[i])
        
    b = []
    for i in range(0, n):
        b.append(f(ts[i]))
        
    cs = find_solution_gauss(np.copy(A), np.copy(b))
    
    res = 0
    for i in range(0, n):
        res += cs[i] * w_i(i)(x)
        
    return res

In [4]:
def print_matrix(A):
    cp_A = np.zeros((len(A), len(A[0])))
    for i in range(0, len(A)):
        for j in range(0, len(A[1])):
            cp_A[i, j] = round(A[i, j], 6)
    print(cp_A)


def print_res(print_a=True, n_range=N_RANGE):
    headers=['\nn', '\ncond(A)', '\nx=-0.5', 'y^n(x)\nx=0', '\nx=0.5', '\nx=-0.5', '|y*(x)-y^n(x)|\nx=0', '\nx=0.5']
    
    ritz_rows = []
    col_rows = []
    
    for n in range(n_range[0], n_range[1] + 1):
        ritz_row = []
        col_row = []
        
        ritz_row.append(str(n))
        col_row.append(str(n))
        
        A = find_A_ritz(n)
        
        if print_a:
            print(f'A for n = {n}:')
            print_matrix(A)
            print()
        
        cond_a = find_cond(np.copy(A))
        
        res_1_ritz = find_solution_ritz(A, -0.5, n)
        res_2_ritz = find_solution_ritz(A, 0, n)
        res_3_ritz = find_solution_ritz(A, 0.5, n)
        
        res_1_col = find_solution_collocation(-0.5, n)
        res_2_col = find_solution_collocation(0, n)
        res_3_col = find_solution_collocation(0.5, n)
        # Calculated using MATLAB 
        y_1_acc = 0.2360
        y_2_acc = 0.5356
        y_3_acc = 0.9645
    
        ritz_row.append(str(cond_a))
        col_row.append(str(cond_a))
        
        ritz_row.append(str(res_1_ritz))
        ritz_row.append(str(res_2_ritz))
        ritz_row.append(str(res_3_ritz))
        col_row.append(str(res_1_col))
        col_row.append(str(res_2_col))
        col_row.append(str(res_3_col))
        
        ritz_row.append(str(np.abs(y_1_acc - res_1_ritz)))
        ritz_row.append(str(np.abs(y_2_acc - res_2_ritz)))
        ritz_row.append(str(np.abs(y_3_acc - res_3_ritz)))
        col_row.append(str(np.abs(y_1_acc - res_1_col)))
        col_row.append(str(np.abs(y_2_acc - res_2_col)))
        col_row.append(str(np.abs(y_3_acc - res_3_col)))
        
        ritz_rows.append(ritz_row)
        col_rows.append(col_row)
        
    print('Метод Ритца')
    print(tabulate(ritz_rows, headers=headers))
    print('\n\nМетод коллокации')
    print(tabulate(col_rows, headers=headers))

In [5]:
print_res()

A for n = 3:
[[ 2.570357 -0.527896 -1.283476]
 [-0.527896  5.811063 -1.429525]
 [-1.283476 -1.429525 10.414828]]

A for n = 4:
[[ 2.570357 -0.527896 -1.283476  0.371753]
 [-0.527896  5.811063 -1.429525 -2.12185 ]
 [-1.283476 -1.429525 10.414828 -2.457543]
 [ 0.371753 -2.12185  -2.457543 15.479588]]

A for n = 5:
[[ 2.570357 -0.527896 -1.283476  0.371753 -0.348663]
 [-0.527896  5.811063 -1.429525 -2.12185   0.696321]
 [-1.283476 -1.429525 10.414828 -2.457543 -3.065146]
 [ 0.371753 -2.12185  -2.457543 15.479588 -3.542101]
 [-0.348663  0.696321 -3.065146 -3.542101 20.830449]]

A for n = 6:
[[ 2.570357 -0.527896 -1.283476  0.371753 -0.348663  0.14714 ]
 [-0.527896  5.811063 -1.429525 -2.12185   0.696321 -0.674294]
 [-1.283476 -1.429525 10.414828 -2.457543 -3.065146  1.033402]
 [ 0.371753 -2.12185  -2.457543 15.479588 -3.542101 -4.072636]
 [-0.348663  0.696321 -3.065146 -3.542101 20.830449 -4.665397]
 [ 0.14714  -0.674294  1.033402 -4.072636 -4.665397 26.373279]]

A for n = 7:
[[ 2.570357 -

In [6]:
print_res(False, (3, 10))

Метод Ритца
                            y^n(x)                         |y*(x)-y^n(x)|
  n    cond(A)    x=-0.5       x=0     x=0.5     x=-0.5               x=0     x=0.5
---  ---------  --------  --------  --------  ---------  ----------------  --------
  3    7.15078  0.328315  0.524413  0.578786  0.0923155       0.0111866    0.385714
  4   11.4132   0.334952  0.521305  0.576203  0.0989519       0.0142946    0.388297
  5   17.0234   0.329469  0.536857  0.567216  0.0934694       0.00125699   0.397284
  6   22.2237   0.329256  0.537671  0.565955  0.0932563       0.00207124   0.398545
  7   28.3083   0.333525  0.534265  0.569522  0.0975248       0.00133538   0.394978
  8   33.9464   0.332618  0.533943  0.570852  0.0966177       0.00165732   0.393648
  9   40.1625   0.333021  0.536354  0.572494  0.0970214       0.000753969  0.392006
 10   46.0892   0.333354  0.536498  0.572261  0.0973539       0.000898483  0.392239


Метод коллокации
                            y^n(x)                     